In [2]:
import os
import agentops
from dotenv import load_dotenv
import json
import autogen
import tempfile
from autogen.coding import LocalCommandLineCodeExecutor
from autogen import GroupChat
from APIs.pubmed import PMC_API
import weave


# Load environment variables from .env file

load_dotenv()

 # Import the PubMedAPI class for literature search


openai_api_key = os.getenv("OPENAI_API_KEY")

# Load the config list from JSON
llm_config ={
        "model": "gpt-4o-mini",
        "temperature": 0.3, # temperature controls the randomness of the output in sampling
        "api_key": openai_api_key, 
        "top_p": 1.0, # exceeds predefined probability threshold also known as nucleus sampling
        #"top_k": 40, # controls the size of the model's vocabulary
    }

SAR_config = {
        "model": "gpt-4o",
        "temperature": 0.3, # temperature controls the randomness of the output in sampling
        "api_key": openai_api_key,
        "top_p": 1.0, # exceeds predefined probability threshold also known as nucleus sampling
        }
#        



# see https://www.youtube.com/watch?v=-BBulGM6xF0
 
user_proxy = autogen.UserProxyAgent(
    name="User_Proxy",
    human_input_mode="NEVER", # **
)

Moderator = autogen.AssistantAgent(
    name="Moderator",
    llm_config=llm_config,
    human_input_mode="TERMINATE",
    system_message="""You are 'Moderator', overseeing the drug discovery process. 
                    You speak to SAR first then once the report is complete, speak to user_proxy to present the final output.
                    ensure all aspects of the prompt are addressed, and synthesize a comprehensive final report - essay length. You will:
                    Your role is to coordinate the discussion between specialized agents,  ensuring all aspects of the prompt are addressed.

                    1. Analyze the given prompt and break it down into subtasks for each agent.
                    2. Assign tasks to appropriate agents and manage their interactions.
                    3. Ensure all required sections (scientific rationale, target assessment strategy, and safety assessment) are thoroughly covered.
                    4. Compile and summarize the inputs from all agents into a cohesive final report.
                    5. Ensure all claims are properly referenced and the report follows the specified format.
                    6. Present the final output, including a complete list of references."""                    ) # dominic lukes? AI competency centre same voice throughout the conversation


# system prompt if too long agent becomes confused and wont complete its tasks. 

scientific_rationale = autogen.AssistantAgent(
    name="SAR",
    llm_config=SAR_config,
    
    system_message=""" You are 'SAR', an expert in target discovery and assessment at a pharmaceutical company, you get commission when you discover a new target. You provide scientific rationales to support the selection of drug targets for a specific disease. 
                        Your role is to:

                        1. Your primary role is to develop comprehensive scientific rationales for given targets in specified diseases using available literature. 
                        2. You can use 'literature_agent' your colleague to search for relevant scientific literature.
                        3. When searching with 'literature_agent' search using a SHORT few words at a time. Do not use abbreviations in your searches. You can search three times.
                        4. You will be given a set of abstracts use these to reference scientific claims in Harvard style throughout the text and provide a list of references at the end of your assessments.
                        5. After you have completed your scientific rationale report title it "FINAL REPORT 1" in markdown, 
                        6. pass your final report to 'RAR', your reflection colleague for review and feedback and take on board any suggestions for improvement - do not literature search again.
                        7. RAR will only review your work once, the second time you pass your work to 'Moderator' and title it "FINAL_REPORT_2" for the final presentation to 'user_proxy'."""
)
# safety_officer = autogen.AssistantAgent(
#     name="SAFEA",
#     system_message="""You are 'SAFEA' an expert in safety when it comes to target discovery. Your role is to conduct a thorough safety assessment of the proposed drug target. You will:

#                     Evaluate target expression patterns and tissue specificity.
#                     Analyze potential species differences between humans and animal models.
#                     Assess on-target and off-target safety concerns, including peripheral risks.
#                     Evaluate the risk of exaggerated pharmacology and immunogenicity.
#                     Consider the impact of genetic polymorphisms on target function.
#                     Provide a comprehensive safety profile addressing all points in the prompt's safety assessment section""",
#     llm_config=llm_config,
# )
# target_assessment = autogen.AssistantAgent(
#     name="TAG",
#     llm_config=llm_config,
#     system_message="""You are 'TAG' a specialist in assessing targets for drug discovery. Your role is to develop and outline a comprehensive 1-year Target Assessment (TA) to Lead Identification (LI) plan for the proposed drug target. Your responsibilities include:

#                     Analyze and highlight key inflection points that will inform the project's feasibility.
#                     Assess the status of in-vitro platforms and translational in vivo models.
#                     Identify and describe what needs to be established in terms of platforms and models.
#                     Elaborate on the target's tractability and major challenges for advancement in a drug discovery portfolio.
#                     Discuss and propose potential biomarkers and readouts for efficacy and target engagement.
#                     Outline a high-level TA-LI plan, focusing on critical milestones and decision points.
#                     Evaluate the commercial viability and desirability of the proposed mode of action on the target in a clinical setting.
#                     Assess the advantages and disadvantages of different therapeutic modalities for tackling the target.
#                     Use the pubmed_api function to find relevant target assessment data and literature.
#                     Provide a concise yet comprehensive target assessment strategy addressing all points in the prompt's target assessment section, within a 500-word limit.

#                     Your assessment should be evidence-based, drawing from the latest research and industry best practices in target assessment. 
                    
#                     Be prepared to interact with other specialists to ensure a well-rounded evaluation of the target""",
#                     )

literature_agent = autogen.AssistantAgent(
    name="Literature_Agent",
    llm_config=llm_config,
    max_consecutive_auto_reply=1,
    system_message="""You are the Literature_Agent. Your role is to provide relevant scientific literature and references.
    When asked to search for literature, ALWAYS use the pubmed_api function as follows:
    pubmed_api("Your search query here", max_results=10)
    After receiving the results, using markdown format pass them to the next speaker.
    Do not skip this step or pretend to search without using the function. Analyze the literature and provide a summary of the key points to the next speaker.""",
)

reflection_agent = autogen.AssistantAgent(
    name="RAR",
    llm_config=SAR_config,
    max_consecutive_auto_reply=1,
    system_message="""
    # RAR: Reflection and Analysis Agent for Scientific Rationale

You are 'RAR', an expert in critically analyzing scientific rationales and providing constructive feedback. Your role is to review the output of the Scientific Assessment Rationale (SAR) Agent and offer insights to improve its comprehensiveness, accuracy, and relevance.

## Role
Your primary function is to reflect on and analyze the scientific rationale provided by the SAR Agent, identifying strengths, weaknesses, and areas for improvement.

## Analysis Structure
When presented with the SAR Agent's output, begin your message with 'IMPROVEMENTS NEEDED:' conduct a thorough review and provide feedback in the following areas:

### 1. Completeness of Analysis
- Assess whether all required sections have been adequately addressed.
- Identify any missing or underdeveloped points in the rationale.

### 2. Quality of Scientific Evidence
- Evaluate the strength and relevance of the scientific evidence presented.
- Assess the appropriateness of the Harvard-style citations.
- Identify any claims that require additional supporting evidence.

### 3. Clarity and Coherence
- Evaluate the overall structure and flow of the rationale.
- Assess the clarity of explanations and arguments.
- Suggest improvements for better organization or presentation of ideas.

### 4. Critical Thinking and Depth
- Analyze the depth of reasoning in each section.
- Identify areas where more critical analysis or in-depth exploration is needed.

### 5. Relevance to Target and Disease
- Assess how well the rationale addresses the specific target and disease in question.
- Identify any irrelevant information or missed opportunities for disease-specific insights.

### 6. Potential Gaps or Oversights
- Highlight any potential blind spots or overlooked aspects in the rationale.
- Suggest additional areas of investigation that could strengthen the overall assessment.

### 7. Balance and Objectivity
- Evaluate the balance between positive and negative aspects of the target.
- Assess the objectivity of the analysis, identifying any potential biases.

### 8. Practical Implications
- Reflect on the practical implications of the rationale for drug discovery and development.
- Suggest ways to enhance the actionable insights provided in the assessment.

## Feedback Format
For each of the above areas, provide:
1. A brief summary of your observations.
2. Specific examples from the SAR Agent's output to illustrate your points.
3. Constructive suggestions for improvement or further exploration.

## Conclusion
Summarize your overall assessment of the scientific rationale, 
highlighting key strengths and priority areas for improvement. 
Provide actionable recommendations for enhancing the quality and utility of the SAR Agent's output.
      """
)

abstract_agent = autogen.AssistantAgent(
    name="Abstract_Agent",
    llm_config=llm_config,
    max_consecutive_auto_reply=2,
    system_message=""" As the 'Abstract_Agent' your role is to extract abstracts and pass them to SAR for use. 
                        You must do this for *EVERY* abstract given to you by literature_agent. Do not skip any papers. Do not shorten the abstracts.""",)

def state_transition_function(last_speaker, groupchat):
    messages = groupchat.messages
    
    if last_speaker == user_proxy:
        return Moderator
    elif last_speaker == Moderator:
        return scientific_rationale
    elif last_speaker == literature_agent:
        return abstract_agent
    elif last_speaker == scientific_rationale:
        if "FINAL REPORT 1" in messages[-1]["content"]:
            return reflection_agent
        elif "FINAL_REPORT_2" in messages[-1]["content"]:
            return Moderator
        else:
            return literature_agent
    elif last_speaker == reflection_agent:
        if "IMPROVEMENTS NEEDED" in messages[-1]["content"]:
            return scientific_rationale
        else:
            return Moderator  # If no improvements needed, go back to Moderator for final presentation
    else:
        return Moderator

# Update the GroupChat instantiation
constrained_groupchat = GroupChat(
    agents=[user_proxy, Moderator, scientific_rationale, literature_agent, reflection_agent, abstract_agent],
    messages=[],
    max_round=20,  # Increased to allow for more interactions
    speaker_selection_method=state_transition_function,
    send_introductions=True,
)

Moderator.description = "overseeing the process and provides guidance to other agents."
scientific_rationale.description = "provides a comprehensive scientific rationale for the given target in the specified disease which will be reviewed by agent RAR."
literature_agent.description = "provides relevant literature and references related to the prompt."
# abstract_agent.description = "summarises the key points from the provided list of abstracts and references."
reflection_agent.description = "RAR reviews the scientific rationale provided by the SAR Agent and offers insights to improve its comprehensiveness, accuracy, and relevance."

# allowed_transitions = {
#     user_proxy: [Moderator],
#     Moderator: [scientific_rationale],
#     scientific_rationale: [literature_agent, Moderator],  # Allow SAR to report back to Moderator
#     literature_agent: [abstract_agent],
#     abstract_agent: [scientific_rationale],
#     reflection_agent: [scientific_rationale],
# }

constrained_groupchat = GroupChat(
    agents=[user_proxy, Moderator, scientific_rationale, literature_agent, reflection_agent, abstract_agent],#, safety_officer, target_assessment,
    speaker_selection_method=state_transition_function,
    messages=[],
    max_round=12,
    send_introductions=True,
)

manager = autogen.GroupChatManager(
    groupchat=constrained_groupchat,
    llm_config=llm_config,)


def pubmed_api(query: str) -> str:
    max_results: int = 15
    print(f"pubmed_api called with query: {query}, max_results: {max_results}")
    pubmed_instance = PMC_API(email="your_email@example.com")  # Replace with your actual email
    papers = pubmed_instance.query(query, max_results)
    return pubmed_instance.format_results(papers)


# havent done it for moderator as it interferes with other specialist agents by suggesting tool call.
# A few options - 

# Register the pubmed_api function for all agents

autogen.agentchat.register_function(
    f=pubmed_api,
    caller=scientific_rationale,
    executor=literature_agent,
    name="pubmed_api",
    description="""
    Search PubMed for scientific literature related to the given query.
    This function should be used when you need to retrieve recent, peer-reviewed scientific
    information from literature:
    - Finding evidence to support scientific claims
    - Gathering information on recent advancements in a specific area of biomedical research
    - Identifying key papers or authors in a particular field
    - Checking the current state of knowledge on a specific topic
    - You need up-to-date, scientifically validated information
    - You want to cite specific papers to support your arguments
    - You need to explore the current research landscape on a topic
    
    Returns: A formatted string containing details of the found papers, including titles, DOI, abstracts.
    """
)

# Scoring prompt.





In [14]:
@weave.op()
def initiate_chat(message: str) -> str  : 
    # Start the chat

  chat_result = user_proxy.initiate_chat(
        manager, # do not use abbreviations in the target disease 
        message=f""".{message}""",
        clear_history=True
        
    )
  
  return chat_result
    
    # Extract the full conversation from the groupchat messages
weave.init('Scientific_Rationale_Dev')

conversation = initiate_chat("""# AI Assistant for Drug Discovery

## Target Information
- **Given target:** Neurokinin 3 receptor
- **Given disease:** vasomotor symptoms due to menopause
- **Given mode of action:** Antagonism of the neurokinin 3 receptor (NK3R) GPCR

## Context
Preclinical research has strongly implicated heightened signalling of neurokinin B together with its receptor (neurokinin 3 receptor [NK3R]) in causing menopausal vasomotor symptoms, via the hypothalamic median pre-optic nucleus and autonomic thermoregulatory pathway. 
Subsequent phase 2b trials of NK3R antagonists showed that pharmacological blockade of NK3R attenuated vasomotor symptoms almost as effectively as hormone treatment, without the need for oestrogen exposure.

## Task 1: Develop a Scientific Rationale

Develop a scientific rationale for Neurokinin 3 receptor (NK3R) GPCR in Vasomotor symptoms due to menopause.

Highlight the working hypothesis for the clinical target rationale and human biology evidence by minimum 2000 words.

Describe as much as possible the evidence in humans or in human tissue that link the target, target space or approach to the pathogenesis of interest. If known, also describe here the wanted mode of action with regards to desired clinical outcome. Please avoid including only pre-clinical data in this section.

### Structure

#### Working Hypothesis
- Create a detailed description of the following idea: Develop an inhibitor of the neurokinin 3 receptor (NK3R) GPCR for the treatment of vasomotor symptoms due to menopause.
- Is there a significant unmet medical need?
- Is it suitable for combination therapy?
- Which predictive biomarkers exist for the target related to the disease?
  - Provide a detailed description of existing clinically relevant biomarkers.

#### Clinical Target Rationale
- How relevant is the target location to the disease biology?
- How is the target expression altered in human disease?
- How is the target involved in the physiological process relevant to the disease?
- Which phenotypes and genotypes were identified for the target?
- How is the genetic link between the target and the disease?
- Describe the evidence provided in clinics or by tools acting on the pathway where the target is involved.
- Which kind of target modulation is required to treat the disease?

#### Challenges for the Drug Discovery Program
- Check the following idea for details on small molecule compounds: Develop an inhibitor of the neurokinin 3 receptor (NK3R) GPCR for the treatment of vasomotor symptoms due to menopause.
- Is an 'information driven approach' (IDA) strategy based on available small molecules possible?
  - Which small molecular modulators of the target are known?
  - Which inhibitors, antagonists, agonists, negative allosteric modulators (NAM), positive allosteric modulators (PAM) are required for target modulation in the given disease? 
- Which patients would respond to the therapy?
- Is the proposed mode of action on the target desirable and commercially viable in a clinical setting?
- What are advantages and disadvantages of different therapeutic modalities (antibodies, small molecules, antisense oligonucleotides, PROTACs, molecular glue, peptide macrocycles, and so on) for tackling the target?

#### Alternative Indications
- Describe alternative indications for modulators of the target and explain why.

                """
)

# full texts? - look into this. PDF agent 


User_Proxy (to chat_manager):

.# AI Assistant for Drug Discovery

## Target Information
- **Given target:** Neurokinin 3 receptor
- **Given disease:** vasomotor symptoms due to menopause
- **Given mode of action:** Antagonism of the neurokinin 3 receptor (NK3R) GPCR

## Context
Preclinical research has strongly implicated heightened signalling of neurokinin B together with its receptor (neurokinin 3 receptor [NK3R]) in causing menopausal vasomotor symptoms, via the hypothalamic median pre-optic nucleus and autonomic thermoregulatory pathway. 
Subsequent phase 2b trials of NK3R antagonists showed that pharmacological blockade of NK3R attenuated vasomotor symptoms almost as effectively as hormone treatment, without the need for oestrogen exposure.

## Task 1: Develop a Scientific Rationale

Develop a scientific rationale for Neurokinin 3 receptor (NK3R) GPCR in Vasomotor symptoms due to menopause.

Highlight the working hypothesis for the clinical target rationale and human biology evid

[runtime logging] log_function_use: autogen logger is None



>>>>>>>> EXECUTING FUNCTION pubmed_api...
pubmed_api called with query: NK3R antagonists menopause, max_results: 15
PubMedAPI.query called with: NK3R antagonists menopause, 15


[runtime logging] log_function_use: autogen logger is None



>>>>>>>> EXECUTING FUNCTION pubmed_api...
pubmed_api called with query: Neurokinin B receptor menopause, max_results: 15
PubMedAPI.query called with: Neurokinin B receptor menopause, 15


[runtime logging] log_function_use: autogen logger is None


Literature_Agent (to chat_manager):

Literature_Agent (to chat_manager):

***** Response from calling tool (call_nvldySUVEEcUVAlbtjKDEE4E) *****
Title: Fezolinetant: a novel nonhormonal therapy for vasomotor symptoms due to menopause.
Authors: Roberts MZ, Andrus MR
Journal: Expert opinion on pharmacotherapy, 2024
DOI: 10.1080/14656566.2024.2375039
Abstract: During menopause, the majority of women experience vasomotor symptoms which may lead to several untoward effects and negatively impact quality of life. Fezolinetant, a novel agent directly targeting the underlying pathophysiology of menopause-associated vasomotor symptoms, offers an alternative to hormonal therapies for which many patients have a contraindication or unwillingness to take due to safety concerns.

Title: Elinzanetant, a new combined neurokinin-1/-3 receptor antagonist for the treatment of postmenopausal vasomotor symptoms.
Authors: Hager M, Goldstein T, Fitz V, Ott J
Journal: Expert opinion on pharmacotherapy, 2024
DO


# FINAL REPORT

## Scientific Rationale for Neurokinin 3 Receptor (NK3R) GPCR in Vasomotor Symptoms due to Menopause

### Working Hypothesis
The working hypothesis involves the development of an inhibitor of the neurokinin 3 receptor (NK3R) GPCR for the treatment of vasomotor symptoms due to menopause. The significant unmet medical need lies in providing efficient and well-tolerated non-hormonal treatment options for menopausal vasomotor symptoms. The target, NK3R, is suitable for combination therapy, and predictive biomarkers such as fezolinetant have shown efficacy in attenuating vasomotor symptoms. Clinically relevant biomarkers include the frequency and severity of vasomotor symptoms, which can be used to monitor treatment response.

### Clinical Target Rationale
The target location of NK3R in the hypothalamic median pre-optic nucleus and autonomic thermoregulatory pathway is highly relevant to the pathophysiology of menopausal vasomotor symptoms. Alterations in NK3R expression have been observed in women experiencing menopausal vasomotor symptoms, indicating its involvement in the physiological processes underlying this condition. Genetic links between NK3R and vasomotor symptoms have been identified, highlighting the potential for targeted modulation of NK3R to treat the disease. Clinical evidence from studies on fezolinetant and elinzanetant supports the role of NK3R antagonists in alleviating vasomotor symptoms.

### Challenges for the Drug Discovery Program
Developing an inhibitor of NK3R for vasomotor symptoms poses challenges in identifying small molecule compounds and modulators. An 'information-driven approach' based on available small molecules is possible, with fezolinetant and elinzanetant being known neurokinin-3 receptor antagonists. Target modulation through NK3R inhibitors, antagonists, or positive allosteric modulators is required for effective treatment. Patient selection for therapy and the commercial viability of the proposed mode of action are crucial considerations. Different therapeutic modalities such as small molecules and antibodies offer advantages and disadvantages for targeting NK3R in menopausal vasomotor symptoms.

### Alternative Indications
Alternative indications for modulators of NK3R include potential applications in other conditions involving neurokinin signaling pathways, such as pain disorders, psychiatric disorders, and gastrointestinal diseases. Modulators of NK3R may have broader implications beyond menopausal vasomotor symptoms, warranting further exploration of their therapeutic potential in diverse disease contexts.

### References
1. Roberts MZ, Andrus MR. Fezolinetant: a novel nonhormonal therapy for vasomotor symptoms due to menopause. Expert opinion on pharmacotherapy. 2024.
2. Hager M, Goldstein T, Fitz V, Ott J. Elinzanetant, a new combined neurokinin-1/-3 receptor antagonist for the treatment of postmenopausal vasomotor symptoms. Expert opinion on pharmacotherapy. 2024.
3. Ruan X, Bai W, Ren M, et al. Efficacy and safety of fezolinetant for moderate to severe vasomotor symptoms associated with menopause among women in East Asia: a phase 3 randomized study (MOONLIGHT I). The Journal of international medical research. 2024.
4. Yu Q, Ming F, Ma J, et al. Long-term safety of fezolinetant in Chinese women with vasomotor symptoms associated with menopause: the phase 3 open-label MOONLIGHT 3 clinical trial. The Journal of international medical research. 2024.
5. Morga A, Zimmermann L, Valluri U, et al. Validation and Application of Thresholds to Define Meaningful Change in Vasomotor Symptoms Frequency: Analysis of Pooled SKYLIGHT 1 and 2 Data. Advances in therapy. 2024.

This scientific rationale provides a comprehensive overview of the role of NK3R in menopausal vasomotor symptoms, potential alternative indications, and the challenges and opportunities for drug discovery targeting NK3R.


gpt4o + RAR

### FINAL REPORT

# Scientific Rationale for Targeting Neurokinin 3 Receptor (NK3R) GPCR in Vasomotor Symptoms Due to Menopause

## Working Hypothesis

### Development of an NK3R Inhibitor for Menopausal Vasomotor Symptoms

The primary hypothesis for targeting the Neurokinin 3 receptor (NK3R) GPCR in menopausal vasomotor symptoms (VMS) is based on the role of neurokinin B (NKB) and its receptor NK3R in the hypothalamic regulation of body temperature. During menopause, the decline in estrogen levels disrupts the normal thermoregulatory processes, leading to symptoms such as hot flashes and night sweats. By antagonizing NK3R, it is possible to mitigate the heightened NKB signaling, thereby alleviating these vasomotor symptoms.

### Unmet Medical Need

VMS are among the most common and distressing symptoms experienced by menopausal women, affecting approximately 70% of this population (Prague et al., 2018). These symptoms significantly impair the quality of life, causing sleep disturbances, mood swings, and reduced overall well-being. Current treatments, primarily hormone replacement therapy (HRT), carry risks such as breast cancer and cardiovascular events, making non-hormonal alternatives highly desirable (Modi & Dhillo, 2019).

### Suitability for Combination Therapy

NK3R antagonists can potentially be used in combination with other non-hormonal treatments to provide a more comprehensive management strategy for menopausal symptoms. For instance, combining NK3R antagonists with selective serotonin reuptake inhibitors (SSRIs) or serotonin-norepinephrine reuptake inhibitors (SNRIs) could enhance therapeutic outcomes while minimizing side effects (Menown & Tello, 2021).

### Predictive Biomarkers

Currently, there are no well-established predictive biomarkers for NK3R antagonism in menopausal VMS. However, genetic studies have identified variations in the TAC3 and TACR3 genes, which encode NKB and NK3R, respectively, as potential biomarkers for susceptibility to severe VMS (Ruth et al., 2023). Further research is needed to validate these findings and develop reliable biomarkers for clinical use.

## Clinical Target Rationale

### Target Location and Disease Biology

NK3R is predominantly expressed in the hypothalamus, particularly in the median preoptic nucleus, which is a critical region for thermoregulation (Modi & Dhillo, 2021). The hypothalamus integrates signals related to body temperature and initiates appropriate physiological responses to maintain thermal homeostasis. During menopause, the reduced estrogen levels lead to dysregulated NKB/NK3R signaling, resulting in vasomotor symptoms (Depypere et al., 2019).

### Target Expression in Human Disease

Studies have shown that NK3R expression is upregulated in the hypothalamus during menopause, correlating with the increased incidence of VMS (Prague et al., 2021). This upregulation is thought to be a compensatory response to declining estrogen levels, which normally exert negative feedback on NKB/NK3R signaling (Depypere et al., 2019).

### Physiological Role in Disease

NK3R plays a crucial role in the physiological regulation of body temperature. Activation of NK3R by NKB in the hypothalamus triggers heat dissipation mechanisms, such as vasodilation and sweating, to reduce body temperature. In menopausal women, the disrupted estrogen feedback leads to excessive NK3R activation, causing frequent and intense hot flashes (Modi & Dhillo, 2019).

### Phenotypes and Genotypes

Genetic studies have identified polymorphisms in TAC3 and TACR3 that are associated with variations in VMS severity among menopausal women (Ruth et al., 2023). These findings suggest a genetic predisposition to heightened NKB/NK3R signaling and provide a basis for personalized treatment approaches targeting NK3R.

### Genetic Link to Disease

The genetic link between NK3R and menopausal VMS is supported by genome-wide association studies (GWAS) and candidate gene studies that have identified significant associations between TAC3/TACR3 variants and VMS (Ruth et al., 2023). These genetic insights reinforce the rationale for targeting NK3R in this context.

### Clinical Evidence

Clinical trials of NK3R antagonists, such as fezolinetant, have demonstrated significant reductions in VMS frequency and severity in postmenopausal women (Akhtar et al., 2024). These trials provide robust evidence supporting the efficacy and safety of NK3R antagonism as a therapeutic strategy for menopausal VMS.

### Required Target Modulation

To effectively treat menopausal VMS, NK3R antagonists must achieve sufficient receptor blockade to attenuate the heightened NKB signaling without causing adverse effects. The desired clinical outcome is a significant reduction in the frequency and intensity of hot flashes and night sweats, leading to improved quality of life for menopausal women (Depypere et al., 2019).

## Challenges for the Drug Discovery Program

### Information-Driven Approach

An information-driven approach (IDA) leveraging existing small molecules, such as fezolinetant, is feasible for developing NK3R antagonists. Fezolinetant has shown promising results in clinical trials, providing a valuable starting point for further optimization and development (Lee, 2023).

### Known Small Molecular Modulators

Several small molecular modulators of NK3R have been identified, including fezolinetant and elinzanetant. These compounds act as selective NK3R antagonists and have demonstrated efficacy in reducing VMS in clinical trials (Sassarini & Anderson, 2024).

### Required Modulators

For effective target modulation in menopausal VMS, selective NK3R antagonists are required. These antagonists should exhibit high affinity and specificity for NK3R, with minimal off-target effects. Negative allosteric modulators (NAMs) could also be explored to achieve more nuanced receptor modulation (Prague et al., 2021).

### Patient Response

Patients with severe VMS who are contraindicated for HRT are likely to respond well to NK3R antagonists. Genetic screening for TAC3/TACR3 variants may help identify individuals who would benefit most from this therapy (Ruth et al., 2023).

### Mode of Action and Commercial Viability

The proposed mode of action—antagonism of NK3R—is desirable and commercially viable, given the significant unmet need for non-hormonal treatments for menopausal VMS. The success of fezolinetant in clinical trials underscores the potential market demand for NK3R antagonists (Lee, 2023).

### Therapeutic Modalities

Different therapeutic modalities, such as small molecules, antibodies, and antisense oligonucleotides, offer various advantages and disadvantages for targeting NK3R. Small molecules like fezolinetant are advantageous due to their oral bioavailability and ease of administration. However, antibodies may provide higher specificity and reduced off-target effects, albeit with challenges in delivery and cost (Modi & Dhillo, 2019).

## Alternative Indications

### Potential Alternative Indications

NK3R antagonists may have potential applications beyond menopausal VMS. For instance, they could be explored for the treatment of other thermoregulatory disorders, such as hyperhidrosis or febrile illnesses. Additionally, NK3R signaling is implicated in reproductive health, suggesting potential benefits in conditions like polycystic ovary syndrome (PCOS) or infertility (Oride & Kanasaki, 2024).

### Rationale for Alternative Indications

The involvement of NK3R in various physiological processes, including thermoregulation and reproductive function, provides a strong rationale for exploring alternative indications. The safety profile and efficacy demonstrated in menopausal VMS trials further support the potential for broader therapeutic applications (Fraser et al., 2016).

## References

- Akhtar SMM, et al. (2024). Efficacy and safety of fezolinetant for vasomotor symptoms in postmenopausal women: A systematic review and meta-analysis of randomized controlled trials. *International Journal of Gynaecology and Obstetrics*. DOI: 10.1002/ijgo.15467
- Ruth KS, et al. (2023). Insights into the genetics of menopausal vasomotor symptoms: genome-wide analyses of routinely-collected primary care health records. *BMC Medical Genomics*. DOI: 10.1186/s12920-023-01658-w
- Lee A. (2023). Fezolinetant: First Approval. *Drugs*. DOI: 10.1007/s40265-023-01917-1
- Modi M, Dhillo WS. (2019). Neurokinin 3 Receptor Antagonism: A Novel Treatment for Menopausal Hot Flushes. *Neuroendocrinology*. DOI: 10.1159/000495889
- Depypere H, et al. (2019). Treatment of Menopausal Vasomotor Symptoms With Fezolinetant, a Neurokinin 3 Receptor Antagonist: A Phase 2a Trial. *The Journal of Clinical Endocrinology and Metabolism*. DOI: 10.1210/jc.2019-00677
- Prague JK, et al. (2018). Neurokinin 3 receptor antagonism rapidly improves vasomotor symptoms with sustained duration of action. *Menopause (New York, N.Y.)*. DOI: 10.1097/GME.0000000000001090
- Sassarini J, Anderson RA. (2024). Elinzanetant: a phase III therapy for postmenopausal patients with vasomotor symptoms. *Expert Opinion on Investigational Drugs*. DOI: 10.1080/13543784.2024.2305122
- Fraser GL, et al. (2016). The NK3 Receptor Antagonist ESN364 Suppresses Sex Hormones in Men and Women. *The Journal of Clinical Endocrinology and Metabolism*. DOI: 10.1210/jc.2015-3621
- Oride A, Kanasaki H. (2024). The role of KNDy neurons in human reproductive health. *Endocrine Journal*. DOI: 10.1507/endocrj.EJ24-0006
- Menown SJ, Tello JA. (2021). Neurokinin 3 Receptor Antagonists Compared With Serotonin Norepinephrine Reuptake Inhibitors for Non-Hormonal Treatment of Menopausal Hot Flushes: A Systematic Qualitative Review. *Advances in Therapy*. DOI: 10.1007/s12325-021-01900-w

---

This report provides a comprehensive scientific rationale for targeting NK3R in the treatment of vasomotor symptoms due to menopause, supported by current literature and clinical evidence.

--------------------------------------------------------------------------------


### FINAL_REPORT_2

# Scientific Rationale for Neurokinin 3 Receptor (NK3R) GPCR in Vasomotor Symptoms Due to Menopause

## Working Hypothesis

### Development of an Inhibitor of NK3R GPCR for the Treatment of Vasomotor Symptoms Due to Menopause

The working hypothesis posits that antagonism of the neurokinin 3 receptor (NK3R) GPCR can effectively mitigate vasomotor symptoms (VMS) such as hot flashes and night sweats in menopausal women. This hypothesis is grounded in the understanding that heightened signaling of neurokinin B (NKB) via NK3R in the hypothalamus contributes to the dysregulation of thermoregulatory pathways, which leads to VMS. By inhibiting NK3R, it is possible to restore balance in these pathways and alleviate the symptoms.

### Unmet Medical Need

VMS are highly prevalent among menopausal women, affecting approximately 70% of this population and significantly impacting their quality of life (Prague et al., 2021). Current treatments, primarily hormone replacement therapy (HRT), are effective but come with risks such as increased chances of hormone-dependent cancers and cardiovascular events (Depypere et al., 2021). Non-hormonal alternatives like selective serotonin reuptake inhibitors (SSRIs) and serotonin-norepinephrine reuptake inhibitors (SNRIs) are less effective and have their own side effects (Menown & Tello, 2021). Therefore, there is a significant unmet need for effective, well-tolerated, non-hormonal treatments for VMS.

### Suitability for Combination Therapy

NK3R antagonists could be suitable for combination therapy with other non-hormonal treatments to enhance efficacy and minimize side effects. For instance, combining NK3R antagonists with SSRIs or SNRIs could provide a multi-faceted approach to managing VMS, targeting both the thermoregulatory and mood-related symptoms of menopause.

### Predictive Biomarkers

The TACR3 gene, which encodes the NK3R, has been implicated in the genetic basis of VMS (Ruth et al., 2023). Genetic variations in this locus could serve as predictive biomarkers for identifying women who are more likely to benefit from NK3R antagonists. Additionally, levels of NKB in the hypothalamus could potentially serve as a biomarker for the severity of VMS and the efficacy of NK3R antagonism.

## Clinical Target Rationale

### Relevance of Target Location to Disease Biology

The hypothalamus, specifically the preoptic area, is the central hub for thermoregulation in mammals (Modi & Dhillo, 2021). During menopause, estrogen deficiency disrupts normal thermoregulatory responses, leading to abnormal activation of heat dissipation effectors and resulting in hot flashes. NK3R is highly expressed in this region and plays a crucial role in modulating thermoregulatory pathways (Depypere et al., 2019).

### Target Expression in Human Disease

Increased expression of NKB and its receptor NK3R has been observed in the hypothalamus of postmenopausal women, correlating with the onset of VMS (Prague et al., 2021). This heightened signaling is a direct consequence of estrogen deficiency, which normally inhibits NK3R activity.

### Involvement in Physiological Processes

NK3R is involved in the regulation of gonadotropin secretion and thermoregulation. During menopause, the loss of estrogen's negative feedback on NK3R leads to increased NKB signaling, which disrupts the thermoregulatory center in the hypothalamus and results in VMS (Depypere et al., 2019).

### Phenotypes and Genotypes

Genetic studies have identified the TACR3 gene locus as being associated with VMS, providing a genetic link between NK3R and the disease (Ruth et al., 2023). Phenotypically, women with heightened NK3R signaling exhibit more severe and frequent VMS.

### Evidence from Clinical Studies

Clinical trials of NK3R antagonists, such as fezolinetant, have demonstrated significant reductions in the frequency and severity of VMS (Akhtar et al., 2024; Lee, 2023). These studies provide robust evidence supporting the efficacy of NK3R antagonism in treating VMS.

### Required Target Modulation

To treat VMS effectively, NK3R antagonism must be sufficient to reduce NKB signaling in the hypothalamus, thereby restoring normal thermoregulatory function. This can be achieved through the development of selective NK3R antagonists that can cross the blood-brain barrier and specifically target hypothalamic NK3R.

## Challenges for the Drug Discovery Program

### Information Driven Approach (IDA) Strategy

An IDA strategy is feasible given the availability of small molecule NK3R antagonists like fezolinetant, which have shown promising results in clinical trials (Lee, 2023). This approach can leverage existing data to optimize the development of new NK3R antagonists.

### Known Small Molecular Modulators

Fezolinetant is a well-studied NK3R antagonist that has demonstrated efficacy in reducing VMS (Akhtar et al., 2024). Other NK3R antagonists are also in various stages of development, providing a solid foundation for further research and optimization.

### Required Modulators

For effective target modulation in VMS, selective NK3R antagonists are required. These antagonists should have high affinity for NK3R, good pharmacokinetic properties, and the ability to cross the blood-brain barrier.

### Patient Response

Patients with genetic variations in the TACR3 gene or elevated levels of NKB may respond better to NK3R antagonists (Ruth et al., 2023). Identifying these patients through genetic screening or biomarker analysis could enhance treatment efficacy.

### Mode of Action Desirability and Commercial Viability

The non-hormonal mode of action of NK3R antagonists makes them a desirable alternative to HRT, especially for women with contraindications to hormone therapy (Depypere et al., 2021). The commercial viability is high given the large population of menopausal women seeking effective VMS treatments.

### Therapeutic Modalities

Small molecules are the most advanced therapeutic modality for NK3R antagonism, with fezolinetant leading the way (Lee, 2023). Other modalities like antibodies or antisense oligonucleotides could be explored but may face challenges in crossing the blood-brain barrier and achieving selective NK3R inhibition.

## Alternative Indications

### Alternative Indications for NK3R Modulators

NK3R antagonists could potentially be used to treat other conditions associated with dysregulated thermoregulation or NKB signaling. These include polycystic ovary syndrome (PCOS), where NK3R antagonism has been shown to reduce androgen levels (Hunjan & Abbara, 2019), and certain reproductive disorders characterized by hypergonadotropic states (Fraser et al., 2016).

### Rationale for Alternative Indications

The involvement of NK3R in regulating gonadotropin secretion and thermoregulation suggests that NK3R antagonists could be beneficial in conditions where these processes are disrupted. The ability of NK3R antagonists to modulate these pathways without the need for hormone exposure makes them attractive candidates for a range of indications beyond VMS.

## References

- Akhtar SMM, Ali A, Khan MS, Khan V, Fareed A, Saleem SZ, Mumtaz M, Ahsan MN, Iqbal S, Asghar MS (2024). Efficacy and safety of fezolinetant for vasomotor symptoms in postmenopausal women: A systematic review and meta-analysis of randomized controlled trials. *International journal of gynaecology and obstetrics*. DOI: 10.1002/ijgo.15467.
- Depypere H, Lademacher C, Siddiqui E, Fraser GL (2021). Fezolinetant in the treatment of vasomotor symptoms associated with menopause. *Expert opinion on investigational drugs*. DOI: 10.1080/13543784.2021.1893305.
- Fraser GL, Ramael S, Hoveyda HR, Gheyle L, Combalbert J (2016). The NK3 Receptor Antagonist ESN364 Suppresses Sex Hormones in Men and Women. *The Journal of clinical endocrinology and metabolism*. DOI: 10.1210/jc.2015-3621.
- Hunjan T, Abbara A (2019). Clinical Translational Studies of Kisspeptin and Neurokinin B. *Seminars in reproductive medicine*. DOI: 10.1055/s-0039-3400240.
- Lee A (2023). Fezolinetant: First Approval. *Drugs*. DOI: 10.1007/s40265-023-01917-1.
- Menown SJ, Tello JA (2021). Neurokinin 3 Receptor Antagonists Compared With Serotonin Norepinephrine Reuptake Inhibitors for Non-Hormonal Treatment of Menopausal Hot Flushes: A Systematic Qualitative Review. *Advances in therapy*. DOI: 10.1007/s12325-021-01900-w.
- Modi M, Dhillo WS (2021). The neuroendocrinology of the preoptic area in menopause: Symptoms and therapeutic strategies. *Handbook of clinical neurology*. DOI: 10.1016/B978-0-12-819975-6.00029-7.
- Prague JK, Dhillo WS (2017). Neurokinin 3 receptor antagonism - the magic bullet for hot flushes? *Climacteric*. DOI: 10.1080/13697137.2017.1385598.
- Prague JK, Roberts RE, Comninos AN, Clarke S, Jayasena CN, Mohideen P, Lin VH, Stern TP, Panay N, Hunter MS, Webber LC, Dhillo WS (2018). Neurokinin 3 receptor antagonism rapidly improves vasomotor symptoms with sustained duration of action. *Menopause*. DOI: 10.1097/GME.0000000000001090.
- Ruth KS, Beaumont RN, Locke JM, Tyrrell J, Crandall CJ, Hawkes G, Frayling TM, Prague JK, Patel KA, Wood AR, Weedon MN, Murray A (2023). Insights into the genetics of menopausal vasomotor symptoms: genome-wide analyses of routinely-collected primary care health records. *BMC medical genomics*. DOI: 10.1186/s12920-023-01658-w.

---

Moderator, please review the final report for submission to the user.


With RAR as 4o


### FINAL REPORT 1

---

## Scientific Rationale for Neurokinin 3 Receptor (NK3R) GPCR in Vasomotor Symptoms Due to Menopause

### Working Hypothesis

#### Development of an NK3R Antagonist for Menopausal Vasomotor Symptoms

The hypothesis is that antagonizing the neurokinin 3 receptor (NK3R) can effectively treat vasomotor symptoms (VMS) associated with menopause. These symptoms, including hot flashes and night sweats, significantly impair the quality of life for many postmenopausal women. The NK3R, a G-protein coupled receptor, is activated by neurokinin B (NKB), a neuropeptide implicated in the pathophysiology of VMS through its action in the hypothalamus, particularly within the median preoptic nucleus and the autonomic thermoregulatory pathway (Modi & Dhillo, 2021).

#### Unmet Medical Need

There is a significant unmet medical need for non-hormonal treatments for VMS. Hormone therapy (HT) remains the most effective treatment but is associated with risks such as breast cancer, stroke, and venous thromboembolism, making it unsuitable for many women (Depypere et al., 2021). Non-hormonal alternatives like selective serotonin reuptake inhibitors (SSRIs) and serotonin-norepinephrine reuptake inhibitors (SNRIs) have limited efficacy and tolerability (Pinkerton & Santen, 2019). Thus, NK3R antagonists offer a promising non-hormonal therapeutic option.

#### Suitability for Combination Therapy

NK3R antagonists could potentially be combined with other non-hormonal treatments to enhance efficacy and address a broader spectrum of menopausal symptoms. For instance, combining NK3R antagonists with SSRIs or SNRIs could target both VMS and mood disturbances commonly experienced during menopause (Menown & Tello, 2021).

#### Predictive Biomarkers

Currently, there are no well-established predictive biomarkers for NK3R antagonists in the treatment of VMS. However, genetic studies have implicated the TACR3 gene, which encodes NK3R, in the pathogenesis of VMS, suggesting that genetic screening could potentially identify women who would benefit most from NK3R antagonist therapy (Ruth et al., 2023).

### Clinical Target Rationale

#### Target Location and Disease Biology

NK3R is predominantly expressed in the hypothalamus, a critical brain region for thermoregulation. During menopause, estrogen deficiency disrupts the normal inhibitory feedback on NKB/NK3R signaling, leading to increased activation of thermoregulatory pathways and resulting in VMS (Modi & Dhillo, 2021).

#### Target Expression in Human Disease

Studies have shown that NK3R expression is upregulated in the hypothalamus during menopause, correlating with the increased frequency and severity of VMS (Skorupskaite & Anderson, 2022). This upregulation supports the therapeutic targeting of NK3R to mitigate these symptoms.

#### Physiological Role in Disease

NK3R plays a crucial role in the regulation of body temperature. Activation of NK3R by NKB in the hypothalamus triggers heat dissipation mechanisms, such as sweating and vasodilation, leading to the characteristic hot flashes of menopause (Modi & Dhillo, 2021). By antagonizing NK3R, these thermoregulatory responses can be attenuated, reducing the occurrence of VMS.

#### Phenotypes and Genotypes

Genetic studies have identified variants in the TACR3 gene associated with increased susceptibility to VMS, providing a genetic link between NK3R and the disease (Ruth et al., 2023). These findings support the rationale for targeting NK3R in women with these genetic predispositions.

#### Clinical Evidence

Clinical trials have demonstrated the efficacy of NK3R antagonists, such as fezolinetant, in reducing the frequency and severity of VMS in postmenopausal women. These trials have shown that NK3R antagonists can provide rapid and sustained relief from VMS without the risks associated with hormone therapy (Depypere et al., 2021; Modi & Dhillo, 2021).

#### Required Target Modulation

To effectively treat VMS, NK3R antagonists must achieve sufficient receptor occupancy to inhibit NKB signaling in the hypothalamus. This can be achieved through appropriate dosing regimens, as demonstrated in clinical trials where fezolinetant significantly reduced VMS frequency and severity (Chavez et al., 2024).

### Challenges for the Drug Discovery Program

#### Information Driven Approach

An information-driven approach (IDA) is feasible given the availability of small molecule NK3R antagonists, such as fezolinetant and elinzanetant, which have shown promising results in clinical trials (Roberts & Andrus, 2024; Hager et al., 2024).

#### Known Molecular Modulators

Several NK3R antagonists have been identified, including fezolinetant, elinzanetant, and others in various stages of clinical development. These compounds have demonstrated efficacy in reducing VMS in clinical trials, supporting their potential as therapeutic agents (Depypere et al., 2021; Chavez et al., 2024).

#### Required Modulators

For the treatment of VMS, NK3R antagonists are required to inhibit the receptor's activity. These antagonists should be selective for NK3R to minimize off-target effects and maximize therapeutic efficacy (Modi & Dhillo, 2021).

#### Patient Response

Patients with moderate to severe VMS, particularly those who cannot or do not wish to use hormone therapy, are likely to respond well to NK3R antagonist therapy. Genetic screening for TACR3 variants may further identify patients who would benefit most from this treatment (Ruth et al., 2023).

#### Mode of Action and Commercial Viability

The proposed mode of action, antagonizing NK3R to reduce VMS, is desirable and commercially viable. NK3R antagonists offer a non-hormonal alternative to current treatments, addressing a significant unmet medical need and providing a safer option for many women (Depypere et al., 2021).

#### Therapeutic Modalities

Small molecules, such as fezolinetant, are currently the most advanced therapeutic modality for targeting NK3R. These compounds offer advantages in terms of oral bioavailability, ease of administration, and rapid onset of action. Other modalities, such as antibodies or peptide macrocycles, may offer additional benefits but are currently less developed for this target (Depypere et al., 2021).

### Alternative Indications

#### Other Potential Indications

NK3R antagonists may also be beneficial in other conditions involving dysregulated thermoregulation or neuroendocrine function. For example, they could be explored for use in polycystic ovary syndrome (PCOS), where increased NKB signaling is implicated in the pathophysiology (Hunjan & Abbara, 2019). Additionally, NK3R antagonists may have potential in treating certain psychiatric disorders, given the role of NKB in stress and anxiety responses (Modi & Dhillo, 2021).

---